# Evaluation of Poincare Embeddings

This notebook demonstrates how well poincare embeddings trained using this [implementation](https://github.com/TatsuyaShirakawa/poincare-embedding) perform on the tasks detailed in the [original paper](https://arxiv.org/pdf/1705.08039.pdf).

This is the list of tasks - 
1. WordNet reconstruction
2. WordNet link prediction
3. Link prediction in collaboration networks
4. Lexical entailment on HyperLex

A more detailed explanation of the tasks and the evaluation methodology is present in the individual evaluation subsections.

## 1. Setup

TODO

## 2. Training
TODO

## 3. Loading the embeddings

### 3.1 C++ embeddings

In [93]:
%load_ext line_profiler
%load_ext snakeviz

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [1]:
% cd ../..

/home/jayant/projects/gensim


In [2]:
embeddings_dir = '/home/jayant/projects/poincare-embedding/work'  # TODO: put model files into repo?

In [101]:
import os

from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from scipy.spatial.distance import euclidean, pdist
from smart_open import smart_open

def transform_cpp_embedding_to_kv(input_file, output_file, encoding='utf8'):
    """Given a C++ embedding tsv filepath, converts it to a KeyedVector-supported file"""
    with smart_open(input_file, 'rb') as f:
        lines = [line.decode(encoding) for line in f]
    if not len(lines):
         raise ValueError("file is empty")
    first_line = lines[0]
    parts = first_line.rstrip().split("\t")
    model_size = len(parts) - 1
    vocab_size = len(lines)
    with open(output_file, 'w') as f:
        f.write('%d %d\n' % (vocab_size, model_size))
        for line in lines:
            f.write(line.replace('\t', ' '))
    
        
class PoincareEmbedding(object):
    """Load and perform distance operations on poincare embeddings"""

    def __init__(self, keyed_vectors):
        """Initialize PoincareEmbeddings via a KeyedVectors instance"""
        self.kv = keyed_vectors
    
    @staticmethod
    def poincare_dist(vector_1, vector_2):
        """Return poincare distance between two vectors"""
        norm_1 = np.linalg.norm(vector_1)
        norm_2 = np.linalg.norm(vector_2)
        euclidean_dist = euclidean(vector_1, vector_2)
        return np.arccosh(
            1 + 2 * (
                (euclidean_dist ** 2) / ((1 - norm_1 ** 2) * (1 - norm_2 ** 2))
            )
        )
        
    @classmethod
    def load_poincare_cpp(cls, input_filename):
        """Load embeddings trained via C++ Poincare model

        Args:
            filepath (str): Path to tsv file containing embeddings

        Returns:
            PoincareEmbedding instance

        """
        keyed_vectors_filename = input_filename + '.kv'
        transform_cpp_embedding_to_kv(input_filename, keyed_vectors_filename)
        keyed_vectors = KeyedVectors.load_word2vec_format(keyed_vectors_filename)
        os.unlink(keyed_vectors_filename)
        return cls(keyed_vectors)
    
    def get_all_distances(self, term):
        """Return distances to all terms for given term, including itself"""
        term_vector = self.kv.word_vec(term)
        all_vectors = self.kv.syn0
        
        euclidean_dists = np.linalg.norm(term_vector - all_vectors, axis=1)
        norm = np.linalg.norm(term_vector)
        all_norms = np.linalg.norm(all_vectors, axis=1)

        return np.arccosh(
            1 + 2 * (
                (euclidean_dists ** 2) / ((1 - norm ** 2) * (1 - all_norms ** 2))
            )
        )
        
    def get_distance(self, term_1, term_2):
        """Returns distance between vectors for input terms

        Args:
            term_1 (str)
            term_2 (str)

        Returns:
            Poincare distance between the two terms (float)
        
        Note:
            Raises KeyError if either term_1 or term_2 is absent from vocabulary

        """
        vector_1, vector_2 = self.kv[term_1], self.kv[term_2]
        return self.poincare_dist(vector_1, vector_2)

In [102]:
filenames = [
#     'wordnet_embeddings_2.tsv',
#     'wordnet_embeddings_5.tsv',
    'wordnet_embeddings_10.tsv',
#     'wordnet_embeddings_20.tsv',
#     'wordnet_embeddings_50.tsv',
#     'wordnet_embeddings_100.tsv',
]
embeddings = {fname: PoincareEmbedding.load_poincare_cpp(os.path.join(embeddings_dir, fname)) for fname in filenames}

In [103]:
test_embedding = embeddings['wordnet_embeddings_10.tsv']

In [107]:
%%time
for i, term in enumerate(test_embedding.kv.vocab.keys(), start=1):
    if i > 10000:
        break
    dists = test_embedding.get_all_distances(term)

/home/jayant/projects/py3/lib/python3.5/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in true_divide
/home/jayant/projects/py3/lib/python3.5/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in true_divide


CPU times: user 39.6 s, sys: 0 ns, total: 39.6 s
Wall time: 39.6 s


## 4. Evaluation

### 4.1 WordNet reconstruction

In [150]:
import csv
from collections import defaultdict
import itertools


class ReconstructionEvaluation(object):
    """Evaluating reconstruction on given network for any embeddings"""
    def __init__(self, filepath, embedding):
        """Initialize evaluation instance with tsv file containing relation pairs
        
        Args:
            filepath (str): path to tsv file containing relation pairs
        
        Returns
            ReconstructionEvaluation instance

        """
        items = set()
        embedding_vocab = embedding.kv.vocab
        positive_relations = defaultdict(set)
        with smart_open(filepath, 'r') as f:
            reader = csv.reader(f, delimiter='\t')
            for row in reader:
                assert len(row) == 2, 'Hypernym pair has more than two items'
                item_1_index = embedding_vocab[row[0]].index
                item_2_index = embedding_vocab[row[1]].index
                positive_relations[item_1_index].add(item_2_index)
                items.update([item_1_index, item_2_index])
        self.items = items
        self.positive_relations = positive_relations
        self.embedding = embedding
    
    
    @staticmethod
    def get_positive_item_ranks(distances, positive_item_indices):
        """Given a numpy array of distances and indices of positive items, compute ranks of positive item distances
        
        Args:
            distances (numpy float array): np array of all distances for a specific item
            positive_item_indices (list): list of indices of positive items
        
        Returns:
            list of ranks of positive items in the same order as `positive_indices`
        """
        positive_item_distances = distances[positive_item_indices]
        negative_item_distances = np.ma.array(distances, mask=False)
        negative_item_distances.mask[positive_item_indices] = True
        # Compute how many negative item distances are less than each positive item distance, plus 1 for rank
        ranks = (negative_item_distances < positive_item_distances[:, np.newaxis]).sum(axis=1) + 1
        return list(ranks) 

    def evaluate_reconstruction(self, embeddings):
        """Evaluate mean rank and MAP for reconstruction for given embeddings
            
        Args:
            embeddings (PoincareEmbedding instance): embeddings for which evaluation is to be done
        
        Returns:
            ??

        """
        ranks = []
        for i, item in enumerate(self.items):
            if not i % 1000:
                print('Evaluating item %d %s' % (i, item))
            if item not in self.positive_relations:
                continue
            positive_items = list(self.positive_relations[item])
            item_term = self.embedding.kv.index2word[item]
            item_distances = self.embedding.get_all_distances(item_term)
            positive_item_ranks = self.get_positive_item_ranks(item_distances, positive_items)
            ranks += positive_item_ranks
            if i > 20000:
                break
        return np.mean(ranks)
        

In [151]:
eval_instance = ReconstructionEvaluation(os.path.join(embeddings_dir, 'wordnet_noun_hypernyms.tsv'), test_embedding)

In [152]:
%%time
print(np.mean(eval_instance.evaluate_reconstruction(test_embedding)))

Evaluating item 0 0


/home/jayant/projects/py3/lib/python3.5/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in true_divide


Evaluating item 1000 1000
Evaluating item 2000 2000
Evaluating item 3000 3000
Evaluating item 4000 4000
Evaluating item 5000 5000
Evaluating item 6000 6000
Evaluating item 7000 7000
Evaluating item 8000 8000
Evaluating item 9000 9000
Evaluating item 10000 10000
Evaluating item 11000 11000
Evaluating item 12000 12000
Evaluating item 13000 13000
Evaluating item 14000 14000
Evaluating item 15000 15000
Evaluating item 16000 16000
Evaluating item 17000 17000
Evaluating item 18000 18000
Evaluating item 19000 19000
Evaluating item 20000 20000
116.01562108
CPU times: user 1min 55s, sys: 4 ms, total: 1min 55s
Wall time: 1min 55s


### 4.2 WordNet link prediction

### 4.3 HyperLex lexical entailment

### 4.1 Link Prediction for collaboration networks